**Concurrency with asyncio**

    Concurrency is about dealing with lots of things at once.
    Parallelism is about doing lots of things at once.
    Not the same, but related.
    One is about structure, one is about execution.
    Concurrency provides a way to structure a solution to solve a problem that may (but not necessarily) be parallelizable.
    
    -- Rob Pike

**Professor Imre Simon liked to say there are tow major sins in science: using different words  to mean the same thing and using one word to mean different things.** If you do any research on concurrent or parallel programming you will find different definitions for "concurrency" and "parallelism".

For real parallelism, you must have multiple cores. A modern laptop has four CPU cores but it running more than 100 processes at any given time under normal. So in practice, most processing happens concurrently and not in parallel. The computer is constantly dealing with 100+ processes, making sure each has an opptotunity to make progress, even if the CPU itself can't do more than four thing at once. That's why Rob Pibke titled tat talkl "Concurrency Is Not Parallelism".

This chapter introduces `asyncio`, a package that implements concurrency with coroutines driven by an event loop. Guido van Rossum developed `asyncio` outside of the Python repository and gave the project a code name of "Tulip".

Tulip was renamed to `asyncio` when it was added to the standard library in Python 3.4. Becuase it uses `yield from` expression extensively, `asyncio` is incompatible with old version of Python.

In this chapter we'll see:

* A comparison between a simple threaded program and the asyncio equivalent, showing the relationship between threads and asynchronous tasks
* How the `asyncio.Future` differs from the `concurrent.futures.Future`
* Asynchronous versions of the flag download examples
* How asynchronous programming manages high concurrency in network applications, without using threads or processes
* How coroutines are a major improvement over callbacks for asynchronous programming
* **How to avoid blocking the event loop by offloading blocking operations to a thread pool**
* Writing `asyncio` servers, and how to rethink web application for high concurrency
* Why `asyncio` is posied to have a big impact in the Python ecosystem.

# Thread Versus Coroutine: A Comparison

In [19]:
import sys
import time
import threading
import itertools

class Signal:
    go = True

def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        time.sleep(.1)
        if not signal.go:
            break
    
    write(' ' * len(status) + '\x08' * len(status))

def slow_function():
    time.sleep(3)
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking', signal))
    print('spinner object:', spinner)
    
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

# result = supervisor()
# print('Answer:', result)

Animating a text spinner with a coroutine

In [32]:
import sys
import asyncio
import itertools


@asyncio.coroutine
def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        
        try:
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    
    write(' ' * len(status) + '\x08' * len(status))

@asyncio.coroutine
def slow_function():
    yield from asyncio.sleep(3)
    return 42

@asyncio.coroutine
def supervisor():
    spinner = asyncio.create_task(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

# loop = asyncio.get_event_loop()
# result = loop.run_until_complete(supervisor())
# loop.close()
# print('Answer:', result)

Here is a summary of the main differences to note between two supervisor impelmentations:

* An `asyncio.Task` is roughly the equivalent of a `threading.Thread`. "a Task like a green thread in libraries that implement cooperative multitasking, such as gevent".
* A Task drives a coroutine, and a Thread invokes a callable.
* You don't instantiate Task objects youself, you get them by passing a coroutine to `loop.create_task(...)`.
* When you get a Task object, it is already scheduled to run; a Thread instance must be explicitly told to run by calling its start method.
* In the threaded supervisor, the `slow_function` is a plain function and is directly invoked by the thread. In the asyncio supervisor, `slow_function` is a courotuine driven by yield from.
* There's no API to terminate a thread from the outside. For tasks, there is the `Task.cancel` instance method, which raises CancelledError inside the coroutine. The coroutine can deal with this by catching the exception in the yield where it's suspended.
* The supervisor coroutine must be executed with loop.run_until_complete in the main function.

One final point related threads veruss coroutine: if you've done any nontrivial programming with threads, you know how challenging it is to reason about the program because the scheduler can interrupt a thread at any time. You must remember to hold locks to protect the critical sections of your program, to avoid getting interrupted in the middle of  a multistep operation - which could leave data in an invalid state.

With coroutines, everything is protected against interruption by default. You must explictly yield to let the rest of the program run. Instead of holding to synchronize the operations of multiple threads, you have coroutine that are "synchronized" by defaultion: only one of them is running at any time. And you want to give up control, you use `yield` or `yield from` to give control back to the scheduler. That's why it is possible to safely cancel a coroutine: a coroutine can only be cancelled when it's suspended at a yeild point, so you can perform cleanup by handling the CancelledError exception.

## asyncio.Future: Nonblocking by Design

The `asyncio.Future` and the `concurrent.futures.Future` classes have mostly the same interface, but are implemented differently.

Futures are created only as the result of scheduling something for execution. In asyncio, `BaseEventLoop.create_task(...)` takes a coroutine, schedules it to run, and returns an asyncio.Task instance - which is also an instance of asyncio.Future, becuase Task is a subclass of Future designed to wrap a coroutine.

Likes its `concurrent.futures.Future`, the `asyncio.Future` class provides `.done()`, `add_done_callback(...)`, and `.result()` methods, but `.result()` is very different. In `asyncio.Future`, the `.result()` methods takes no arguments, so you can't specify a timeout. Also, if you call `.result()` and the future is not done, it does not block waiting for the result. Instead, an `asyncio.InvalidStateError` is raised.

Using `yield from` with a future automatically takes care of waiting for it to finish, without blocking the event loop - because in asyncio, yield from is used to give control back to the event loop.

Note that using yield from with a future is the coroutine equivalent of the functionality offered by `add_done_callback`: instead of triggering a callback, when the delayed operation is done, the event loop sets the result of the future, and the `yield from` expression produces a return value inside our suspended coroutine, allowing ti to resume.

In summary, because `asyncio.Future` is designed to work with yield form, these methods are often not needed:

* **You don't need `my_future.add_done_callback(...)` because you simply put whatever processing you would do after the future is done in the lines that follow `yield from my_future` in your coroutine. That's the big advantage of having coroutines: functions that can be suspeded and resumed.**

* **You don't need `my_future.result()` because the value of a yield from  expression on a future is the result(e.g., result = yield from my_future)**



## Yield from Futures, Takss, and Coroutines

In asyncio, there is a close relationship between futures and coroutines because you can get result of an asyncio.Future by yielding from it. This means that `res = yield from foo()` works if foo is a coroutine function or if foo is plain function that return a Future or Task intance.

In order to execute, a coroutine must be scheduled, and then it's wrapped in an asyncio.Task. 

# Downloading with asynio and aiohttp

In [47]:
import asyncio
import aiohttp

url_list = ['https://www.baidu.com'] * 10

@asyncio.coroutine
def download_one(url):
    # blocking operations are implemented as coroutine, and your
    # code delegates to them via yield from so they run asynchronously.
    resp = yield from aiohttp.request('GET', url)
    resp_content = yield from resp.read()
    return resp_content

def download_many(url_list):
    # get a reference to the underlying event-loop implementation
    loop = asyncio.get_event_loop()

    # Build a list generator objects by calling the download_one
    todo = [download_one(url) for url in url_list]
    
    # Despite its name, wait is not a blocking function.
    wait_coro = asyncio.wait(todo)
    
    # Execute the event loop until wait_coro is done, this is
    # where the script will block while the event loop runs.
    res, _ = loop.run_until_complete(wait_coro)
    
    # shut down the event loop.
    loop.close()
    
    return len(res)

The `asyncio.wait(...)` coroutine accepts an iterable of futures or coroutiness; wait wraps each coroutine in a Task. The end result is that all objects managed by wait become instance of Future. Because it is a coroutine function, calling `wait(...)` returns a coroutine/generator object; this is what the `wait_coro` variable holds. To drive the coroutine, we pass it to `loop.run_until_complete(...)`.

To summarize: as we use asyncio, our asynchronous code consits of coroutines that are delegating generators driven by asyncio itself and that delegate to asyncio library coroutines - possibly by way of some third-party library such as aiohttp. This arrangement creats pipelines where the asyncio event loop drives - through our coroutine - the library functions that perform the low-level asynchronous I/O.

# Running Circling Around Blocking Calls

Callbacks are the traditional way to implement asychronous calls with low memory overhead. They are a low-level concept, similar to the oldest and most primitive concurrency mechanism  of all: hardware interrupts. Instead of wating for a response, we register a function  to be called when something happens. In this way, every call we make can be nonblocking.

**When used as coroutine, generators provide an alternative way to do asychronous programming. From the perspective of the event loop, invoking a callback or calling `.send()` on a suspended conroutine is pretty much the same. There is a memory overhead for each suspended coroutine, but it's orders of manitude smaller than the overhead for each thread. And they avoid the dreaded "callback hell".**

As the asyncio infrastruture gets the first response back, the event loop sends it to the waiting `get_flag` coroutine. As `get_flags` get response, it advances to the next yield from, which calls `resp.read()` and yields control back to the main loop. Other reponses arrive in close succession. As each get_flag returns, the delegating generator download_flag resumes and save the image file.

# From Callbacks to Futures and Coroutines

Event-oriented programming with coroutins requires some effort to master.

Anyone with some experience in callback-style event-oriented programming knowns the term "callback hell": the nesting of callbacks  when one operation depends on the result of the previous operation.

# Writing asyncio Servers

## An asyncio TCP Sever

## An aiohttp Web Server

## Smater Clients for Better Concurrency